# **Fetching Census data**
Author: Liubov Dumarevskaya

GitHub page of the author: https://liubovd.github.io/

Last Updated: June 13, 2025


Description: This is the workflow aimed on fetching ASC data for Rhode Island per tract. You can choose different vairables and years.



**Make sure you run all the cells in the order!**

### **Run the next cell to download library of available variables:**

In [ ]:
import pandas as pd
url = 'https://raw.githubusercontent.com/LiubovD/liubovd.github.io/refs/heads/main/workshops/table_asc_for_workflow_modified.csv'
df = pd.read_csv(url)
my_dict = dict(zip(df.iloc[:, 1], df.iloc[:, 0]))
print(my_dict)

### **Select variable to map:**

In [ ]:
import ipywidgets as widgets
from IPython.display import display

dropdown = widgets.Dropdown(
    options=[(desk, code) for desk, code in my_dict.items()],
    description='Variable:',
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='80%')
)

display(dropdown)

### **Set the desired year (2020-2023) and double-check your choosen parameters .**

In [ ]:
variable = dropdown.value
year = input("Enter the year: ")
url = f"https://api.census.gov/data/{year}/acs/acs5"
print("Constructed URL:", url)

params = {
    "get": f"NAME,{variable}",
    "for": "tract:*",
    "in": "state:44"  # Rhode Island
}

print("Constructed parameters:", params)

### **Request data and check your dataset. Some of the variables are not available for all years for RI on the tract level, in this case you will get either None or -666666 value in the table.**

In [ ]:
import requests
response = requests.get(url, params=params)
data = response.json()

df = pd.DataFrame(data[1:], columns=data[0])
print(df.head())

### **Add GEOID field which contains full tract information and allows to join table to map:**

In [ ]:
df['GEOID'] = df['state'] + df['county'] + df['tract']
print(df)

### **Get information for population:**

In [ ]:
url_population = f"https://api.census.gov/data/{year}/acs/acs5?get=NAME,B01001_001E&for=tract:*&in=state:44"

response = requests.get(url_population)
pop_data = response.json()

df_pop = pd.DataFrame(pop_data[1:], columns=pop_data[0])\

print(df_pop.head())

### **Add population numbers to the table:**

In [ ]:
df = df.merge(df_pop[['NAME', 'B01001_001E']], on='NAME', how='left')
df = df.rename(columns={'B01001_001E': 'population'})
print(df)

In [ ]:
df[variable] = pd.to_numeric(df[variable], errors='coerce')
df['population'] = pd.to_numeric(df['population'], errors='coerce')
df['rate'] = (df[variable]/df['population'])
print(df.head())

### **Create a CSV table in your virtual Colab environment:**

In [ ]:
df.to_csv(f"ASC_RI_{variable}_{year}.csv", index=False)

### **Download the table to your computer.**

In [ ]:
from google.colab import files
files.download(f"ASC_RI_{variable}_{year}.csv")

### **Skip next cell**

In [ ]:
import geopandas as gpd

input_file = "https://liubovd.github.io/maps/ri_tract_with_area.geojson"
gdf = gpd.read_file(input_file)

merged_gdf = gdf.merge(df, left_on='GEOID', right_on='GEOID', how='left')
merged_gdf = merged_gdf.drop_duplicates(subset=['GEOID'])
merged_gdf[variable] = pd.to_numeric(merged_gdf[variable], errors='coerce')
merged_gdf["ALAND"] = pd.to_numeric(merged_gdf["ALAND"], errors='coerce')
merged_gdf["rate"] = (merged_gdf[variable] / merged_gdf["ALAND"])*1000000
print(merged_gdf.head())
output_file = "merged_tracts_RI.geojson"

print(f"\nSuccessfully saved merged data to {output_file}")

### **Create a base map with tract polygons**

Choose whether you want to use rate per 100 000 people or direct number.

In [ ]:
import ipywidgets as widgets
from IPython.display import display
mapping_choice = widgets.ToggleButtons(
    options=['Map rate',  'Map value'],
    description='Choose:',
    disabled=False,
    button_style='',
    tooltips=['Description of slow', 'Description of fast'],
)
display(mapping_choice)

In [ ]:
import folium

if mapping_choice.value == 'Map rate':
  value_to_map = "rate"
else:
  value_to_map = variable


m = folium.Map(location=[41.58, -71.47], zoom_start=9)
geojson_path = "https://liubovd.github.io/maps/ri_tract_with_muni.geojson"
folium.GeoJson(geojson_path).add_to(m)

folium.Choropleth(
    geo_data=geojson_path,
    name="Choropleth",
    data=df,
    columns=["GEOID", value_to_map],
    key_on="feature.properties.GEOID",
    fill_color="YlOrRd",
    fill_opacity=0.7,
    line_opacity=0.2,
    legend_name=f"{variable}_per_total population",
    highlight=True
).add_to(m)

m